In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
import os
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Flatten, concatenate, Reshape
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf


In [3]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        if path.endswith(".jpg") or path.endswith(".png"):
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, (224, 224))  # Resize images to a common size
            images.append(img)
            labels.append(label)
    return images, labels

In [4]:
def load_dataset(data_folder):
    images = []
    labels = []
    for label in os.listdir(data_folder):
        label_folder = os.path.join(data_folder, label)
        if os.path.isdir(label_folder):
            label_images, label_labels = load_images_from_folder(label_folder, label)
            images.extend(label_images)
            labels.extend(label_labels)
    return np.array(images), np.array(labels)

In [36]:
# from sklearn.utils import shuffle

# Load the dataset

train_folder = '/Users/mac/Documents/clonerepo/Deep-Fake-Model/dataset/train/'
val_folder = '/Users/mac/Documents/clonerepo/Deep-Fake-Model/dataset/validation/'
test_folder = '/Users/mac/Documents/clonerepo/Deep-Fake-Model/dataset/test/'

train_images, train_labels = load_dataset(train_folder)
val_images, val_labels = load_dataset(val_folder)
test_images, test_labels = load_dataset(test_folder)



In [37]:
train_images

array([[[[ 43,  35,  24],
         [ 43,  35,  24],
         [ 43,  35,  24],
         ...,
         [ 42,  37,  23],
         [ 42,  37,  23],
         [ 42,  37,  23]],

        [[ 43,  35,  24],
         [ 43,  35,  24],
         [ 43,  35,  24],
         ...,
         [ 42,  37,  23],
         [ 44,  36,  23],
         [ 44,  36,  23]],

        [[ 43,  35,  24],
         [ 43,  35,  24],
         [ 43,  35,  24],
         ...,
         [ 42,  37,  23],
         [ 44,  36,  23],
         [ 44,  36,  23]],

        ...,

        [[ 33,  32,  21],
         [ 33,  32,  23],
         [ 33,  32,  23],
         ...,
         [ 35,  29,  20],
         [ 35,  28,  18],
         [ 35,  28,  18]],

        [[ 33,  32,  25],
         [ 33,  32,  25],
         [ 33,  32,  25],
         ...,
         [ 33,  29,  23],
         [ 34,  28,  21],
         [ 34,  28,  21]],

        [[ 33,  32,  25],
         [ 33,  32,  25],
         [ 33,  32,  25],
         ...,
         [ 33,  29,  23],
        

In [38]:
train_labels

array(['real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
      

In [39]:
# Shuffle the training set
# train_images, train_labels = shuffle(train_images, train_labels, random_state=42)

# Normalize pixel values
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0



In [40]:
train_images

array([[[[0.16862745, 0.1372549 , 0.09411765],
         [0.16862745, 0.1372549 , 0.09411765],
         [0.16862745, 0.1372549 , 0.09411765],
         ...,
         [0.16470588, 0.14509804, 0.09019608],
         [0.16470588, 0.14509804, 0.09019608],
         [0.16470588, 0.14509804, 0.09019608]],

        [[0.16862745, 0.1372549 , 0.09411765],
         [0.16862745, 0.1372549 , 0.09411765],
         [0.16862745, 0.1372549 , 0.09411765],
         ...,
         [0.16470588, 0.14509804, 0.09019608],
         [0.17254902, 0.14117647, 0.09019608],
         [0.17254902, 0.14117647, 0.09019608]],

        [[0.16862745, 0.1372549 , 0.09411765],
         [0.16862745, 0.1372549 , 0.09411765],
         [0.16862745, 0.1372549 , 0.09411765],
         ...,
         [0.16470588, 0.14509804, 0.09019608],
         [0.17254902, 0.14117647, 0.09019608],
         [0.17254902, 0.14117647, 0.09019608]],

        ...,

        [[0.12941176, 0.1254902 , 0.08235294],
         [0.12941176, 0.1254902 , 0.09019608]

In [41]:
# Convert labels to numerical format
label_encoder = LabelEncoder()
y_train_encoded = to_categorical(label_encoder.fit_transform(train_labels))
y_test_encoded = to_categorical(label_encoder.transform(test_labels))
y_val_encoded = to_categorical(label_encoder.transform(val_labels))

In [42]:
y_test_encoded

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

# Training with CNN VGG16 Model

In [86]:
from tensorflow.keras.applications import VGG16
# Define VGG16 model
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze layers to prevent training on ImageNet weights
for layer in vgg_model.layers:
    layer.trainable = False

# Flatten VGG output
flattened_vgg_output = Flatten()(vgg_model.output)

# Add dense layers for classification
x = Dense(128, activation='relu')(flattened_vgg_output)
output = Dense(num_classes, activation='softmax')(x)

# Create the final model
image_only_model = Model(inputs=vgg_model.input, outputs=output)

from tensorflow.keras.optimizers.schedules import ExponentialDecay

initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=1000, decay_rate=0.9)


# Compile the model
image_only_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10  # You can adjust the number of epochs as needed
batch_size = 32  # You can adjust the batch size as needed



# Rest of your code...

# Train the model
image_only_model.fit(
    train_images,
    y_train_encoded,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(val_images, y_val_encoded)
)

# Evaluate the model
test_loss, test_accuracy = image_only_model.evaluate(test_images, y_test_encoded)
print(f'Test Accuracy: {test_accuracy}')


Epoch 1/10
13/13 [==============================] - 58s 5s/step - loss: 8.8385 - accuracy: 0.8292 - val_loss: 0.0258 - val_accuracy: 0.9932
Epoch 2/10
13/13 [==============================] - 58s 5s/step - loss: 0.1469 - accuracy: 0.9950 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
13/13 [==============================] - 55s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0299 - val_accuracy: 0.9865
Epoch 4/10
13/13 [==============================] - 56s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.4927 - val_accuracy: 0.8716
Epoch 5/10
13/13 [==============================] - 55s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.7853 - val_accuracy: 0.8378
Epoch 6/10
13/13 [==============================] - 57s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.8810 - val_accuracy: 0.8108
Epoch 7/10
13/13 [==============================] - 58s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.9085 - val_accuracy: 0.8

## Prediction on a new dataset

In [67]:
new_data= '/Users/mac/Documents/clonerepo/Deep-Fake-Model/modi/dataset/test/'
newdata_images, new_data_labels = load_dataset(new_data)

#predictions = image_only_model.predict(new_data)


In [68]:
new_data_labels

array(['real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake'], dtype='<U4')

In [69]:
newdata_images = newdata_images / 255.0
newdata_images

array([[[[0.45098039, 0.34117647, 0.32156863],
         [0.43921569, 0.3372549 , 0.30196078],
         [0.43529412, 0.32941176, 0.28627451],
         ...,
         [0.78039216, 0.71764706, 0.60392157],
         [0.77254902, 0.70980392, 0.59215686],
         [0.76470588, 0.70196078, 0.57254902]],

        [[0.44313725, 0.33333333, 0.31764706],
         [0.44313725, 0.3372549 , 0.30588235],
         [0.44313725, 0.3372549 , 0.29019608],
         ...,
         [0.78823529, 0.72941176, 0.61568627],
         [0.78431373, 0.7254902 , 0.60392157],
         [0.77647059, 0.71372549, 0.58823529]],

        [[0.42745098, 0.31764706, 0.30196078],
         [0.43529412, 0.32941176, 0.29411765],
         [0.44313725, 0.3372549 , 0.29019608],
         ...,
         [0.78823529, 0.72941176, 0.61176471],
         [0.79607843, 0.73333333, 0.62352941],
         [0.79215686, 0.7254902 , 0.61568627]],

        ...,

        [[0.51372549, 0.37647059, 0.36078431],
         [0.50980392, 0.35686275, 0.32941176]

In [70]:
predictions = image_only_model.predict(newdata_images)


3/3 [==============================] - 7s 2s/step


In [71]:
predictions

array([[9.24201668e-05, 9.99907553e-01],
       [1.20811033e-04, 9.99879062e-01],
       [4.99780738e-11, 9.99999940e-01],
       [2.73550256e-07, 9.99999702e-01],
       [2.18488228e-09, 9.99999940e-01],
       [2.67098835e-06, 9.99997318e-01],
       [6.69304371e-01, 3.30695480e-01],
       [2.14199876e-08, 9.99999940e-01],
       [3.10868842e-07, 9.99999583e-01],
       [5.71652026e-06, 9.99994218e-01],
       [7.55303446e-08, 9.99999821e-01],
       [1.69131876e-04, 9.99830961e-01],
       [1.52998192e-09, 9.99999940e-01],
       [1.06875619e-08, 9.99999940e-01],
       [1.01866185e-07, 9.99999821e-01],
       [4.98422969e-06, 9.99994934e-01],
       [5.78306015e-07, 9.99999344e-01],
       [9.09797848e-10, 9.99999940e-01],
       [9.44670209e-09, 9.99999940e-01],
       [1.34913847e-07, 9.99999821e-01],
       [7.09513233e-06, 9.99992788e-01],
       [8.15143250e-03, 9.91848528e-01],
       [2.11995371e-07, 9.99999702e-01],
       [1.64720780e-06, 9.99998271e-01],
       [3.255154

In [72]:
# Example: Binary classification
threshold = 0.5  # Adjust the threshold based on your needs

binary_predictions = (predictions > threshold).astype(int)


In [73]:
binary_predictions

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0,

In [77]:
# Map binary labels to 'Real' or 'Fake'
class_labels = ['Real', 'Fake']
mapped_predictions = [class_labels[prediction] for prediction in binary_predictions.flatten()]


In [78]:
mapped_predictions

['Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Fake',
 'Real',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Fake',
 'Real',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 

## Prediction on a single data image

In [90]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

# Load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img_to_array(img)  # Convert to NumPy array
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Specify the path to your new image
new_image_path = "/Users/mac/Documents/clonerepo/Deep-Fake-Model/modi/dataset/test/real/modi4.jpg"

# Load and preprocess the new image
new_image = load_and_preprocess_image(new_image_path)

# Make predictions
predictions = image_only_model.predict(new_image)

# Assuming binary classification, you can map predictions to 'Real' or 'Fake' based on a threshold
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)
class_labels = ['Real', 'Fake']

mapped_prediction = class_labels[binary_predictions[0][0]]

print(f'The predicted label for the new image is: {mapped_prediction}')


1/1 [==============================] - 1s 818ms/step
The predicted label for the new image is: Real


In [118]:
# Save the model
model.save("deepfake_detection_model_vgg16.h5")

#from tensorflow.keras.models import load_model

# Load the saved model
#loaded_model = load_model("deepfake_detection_model.h5")


## Objective:
The objective of this deep learning project is to develop a robust system for detecting DeepFake images, which are manipulated images generated by AI algorithms to deceive viewers. The project focuses on binary classification, distinguishing between real and manipulated images.

Key Components:

Dataset: A curated dataset containing a mix of real and DeepFake images is used for model training and evaluation. Each image is labeled as 'Real' or 'Fake.'

Model Architecture: The project employs a convolutional neural network (CNN) architecture, specifically VGG16, known for its effectiveness in image classification tasks. Transfer learning is utilized by leveraging pre-trained weights on a large dataset.

Data Preprocessing: Images are preprocessed by resizing, normalizing pixel values, and augmenting the dataset with techniques like rotation and flipping to enhance model generalization.


Regularization Techniques: Dropout layers and batch normalization are incorporated to reduce overfitting and stabilize training.

Learning Rate Scheduler: An exponential decay learning rate scheduler is implemented to dynamically adjust the learning rate during training.

Evaluation Metrics: Model performance is evaluated using metrics such as accuracy, precision, recall, and F1 score. 

Results:
The final model achieves a test accuracy of 95%, demonstrating its effectiveness in distinguishing between real and manipulated images. Precision and recall scores are balanced, ensuring a reliable detection system.

